In [64]:
import time
import konlpy
import openpyxl
import re
import pandas as pd
import requests
from konlpy.tag import Kkma
from selenium import webdriver
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup



In [62]:
kma = Kkma()

In [63]:
wb = openpyxl.Workbook()
sheet = wb.active


# 에이전트+크롬디버깅모드 + 로그인 + 자게이동 + 다음 페이지 이동 코드
# @@@@절대 건들지마@@@@
# +개발자 도구 킨 상태로 실행 불가
# +자게 페이지 총 28408 개, 현재시점 5년까지만 존재

In [79]:

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_driver = "/usr/local/bin/chromedriver" # Your Chrome Driver path

#chrome_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)  # Your Chrome Driver path

driver = webdriver.Chrome(chrome_driver, options=chrome_options)

id = "aaa4"
pw = "1234"
driver.get("https://everytime.kr")
time.sleep(1)


# id_input = (driver.find_element(By.XPATH,'//*[@id="container"]/form/p[1]/input')).click() #아이디


driver.find_element(By.XPATH,'/html/body/aside/div[1]/a[2]').click() # 로그인 이동 버튼
time.sleep(2)

driver.find_element(By.NAME, 'userid').send_keys(id) # 아이디
time.sleep(2)

driver.find_element(By.NAME, 'password').send_keys(pw) #비밀번호 
time.sleep(2)

driver.find_element(By.XPATH, '//*[@id="container"]/form/p[3]/input').click() # 로그인 확인 버튼 
time.sleep(1)


driver.find_element(By.XPATH, '//*[@id="submenu"]/div/div[2]/ul/li[1]/a').click() # 자유게시판 이동

time.sleep(3)

# driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 다음 페이지 이동




/Users/KBS/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


# 첫 번째 크롤링 방법


In [ ]:
# every_url = 'http://everytime.kr/login'

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_driver = "/usr/local/bin/chromedriver" # Your Chrome Driver path

#chrome_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)  # Your Chrome Driver path

driver = webdriver.Chrome(chrome_driver, options=chrome_options)

id = "aaa4"
pw = "1234"
driver.get("https://everytime.kr")
time.sleep(1)


# id_input = (driver.find_element(By.XPATH,'//*[@id="container"]/form/p[1]/input')).click() #아이디


driver.find_element(By.XPATH,'/html/body/aside/div[1]/a[2]').click() # 로그인 이동 버튼
time.sleep(2)

driver.find_element(By.NAME, 'userid').send_keys(id) # 아이디
time.sleep(2)

driver.find_element(By.NAME, 'password').send_keys(pw) #비밀번호 
time.sleep(2)

driver.find_element(By.XPATH, '//*[@id="container"]/form/p[3]/input').click() # 로그인 확인 버튼 
time.sleep(1)


driver.find_element(By.XPATH, '//*[@id="submenu"]/div/div[2]/ul/li[1]/a').click() # 자유게시판 이동

time.sleep(3)

# driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 다음 페이지 이동


# result = []
# cnt = 0 # 첫번ㅈ째 페이지부터 순환하기 위함

# # 첫 번째 페이지가기

# while True:
#     print('page'+str(cnt))
#     if cnt > 300 : 
#         break
    
#     cnt = cnt + 1
#     driver.get('http://everytime.kr/${게시판넘버(학교마다 다름)}/p/')+ str(cnt)
#     driver.implicitly_wait(1)
    
#     # 게시글 링크가져오기?
#     post = driver.find_elements_by_css_selector('article > article')
#     links = [post.get_attribute('href') for post in posts]
    
#     # 게시글 세부내용
#     for link in links:
#         driver.get(link)
        
#         comments = driver.find_elements_by_css_selector('p.large')
        
#         for comment in comments:
#             results.append(comment.text)




# 두 번째 빵법


In [87]:
# every_url = 'http://everytime.kr/login'

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_driver = "/usr/local/bin/chromedriver" # Your Chrome Driver path

#chrome_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)  # Your Chrome Driver path

driver = webdriver.Chrome(chrome_driver, options=chrome_options)

id = "aaa4"
pw = "1234"
driver.get("https://everytime.kr")
time.sleep(1)


# id_input = (driver.find_element(By.XPATH,'//*[@id="container"]/form/p[1]/input')).click() #아이디




driver.find_element(By.XPATH,'/html/body/aside/div[1]/a[2]').click() # 로그인 이동 버튼
time.sleep(2)

driver.find_element(By.NAME, 'userid').send_keys(id) # 아이디
time.sleep(2)

driver.find_element(By.NAME, 'password').send_keys(pw) #비밀번호 
time.sleep(2)

driver.find_element(By.XPATH, '//*[@id="container"]/form/p[3]/input').click() # 로그인 확인 버튼 
time.sleep(1)


driver.find_element(By.XPATH, '//*[@id="submenu"]/div/div[2]/ul/li[1]/a').click() # 자유게시판 이동

time.sleep(3)

# driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 다음 페이지 이동



## 크롤링 함수 선언

title_data = []
content_data = []
vote_data = []
comment_data = []

def crawling(pg):
    soup = BeautifulSoup(pg,"html.parser")
    to_body = soup.find(class_="wrap articles")
    title = to_body.find_all("h2")
    content = to_body.find_all("p")
    vote = to_body.find_all(class_="vote")
    comment = to_body.find_all(class_="comment")

    for i in range(0,len(title)):
        title_data.append(title[i].text)
        content_data.append(content[i].text)
        vote_data.append(vote[i].text)
        comment_data.append(comment[i].text)

    return title_data, content_data, vote_data, comment_data

# 6. 크롤링 시작

page_num = 1
title, content, vote, comment = crawling(driver.page_source)
print("[INFO] page %d completed ... "%page_num)

while True:
    page_num += 1
    try:   
        #1 driver.find_element_by_class_name("next").click() # 다음페이지 이동?
        #2 driver.find_element(By.class_name,"next").click() # 새버전 클래스
        # driver.find_element(By.XPATH, "//*[@id="container"]/div[2]/div[2]/a").click() #ㄷ쌍다옴표 색 없음
        driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 맞나?
        time.sleep(2)
        title, content, vote, comment = crawling(driver.page_source)
        print("[INFO] page %d completed ... "%page_num)

    except:
        try:
            time.sleep(5)
            #기존 코드 driver.find_element_by_class_name("next").click()
            driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[2]/a').click() # 맞나?
            time.sleep(1)
            title, content, vote, comment = crawling(driver.page_source)
            printprint("[INFO] page %d completed ... "%page_num)
        except:
            print("[INFO] page %d is the last page. nice work!"%page_num)
            break
        
# 7 데이터프레임으로 변환 , 시각화

every_time_dict = {"title": title, "content": content, "vote": vote, "comment": comment}
every_time_dataframe = pd.DataFrame(every_time_dict)
display(every_time_dataframe)

# 8 csv format으로 저장

every_time_dataframe.to_csv('/Users/KBS/Documents/CNU_Everytime_SentimentAnalysis/kbs/data\data.csv', index=False, header = True)



/Users/KBS/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


[INFO] page 1 completed ... 
[INFO] page 2 completed ... 
[INFO] page 3 completed ... 
[INFO] page 4 completed ... 
[INFO] page 5 completed ... 
[INFO] page 6 completed ... 
[INFO] page 7 completed ... 
[INFO] page 8 completed ... 
[INFO] page 9 completed ... 
[INFO] page 10 completed ... 
[INFO] page 11 completed ... 
[INFO] page 12 completed ... 
[INFO] page 13 completed ... 
[INFO] page 14 completed ... 
[INFO] page 15 completed ... 
[INFO] page 16 completed ... 
[INFO] page 17 is the last page. nice work!


,title,content,vote,comment
0,건전 마사지샵 카운터 알바,ㄱㅊ? 부모님한테 뭐라 들을 가능성 없겠지? 친구는 좀 놀릴 것 같고,0,2
1,원가회계 질문,여기서 단위당 변동원가를 어떻게 구하나요?,0,0
2,경영 시험기간에 확진 되어본 분,시험 어떻게 해주셨어요..? ㅜㅠ,0,0
3,경영과학 시험 봤던 사람,수업시간에 한 문제만 나와?? 아님 수업 때 안한 책에 있는 문제들로 시험 나와??,0,1
4,원가회계 질문,밑줄 친부분이 이해가 안가서요..분자는 매출원가여야 하지 않나요?,0,2
...,...,...,...,...
295,충전기,혹시 스타벅스 장대점에 계신분 c타입 충전기 빌려주실분 있나여…. 없겠져….ㅠㅠㅠㅠ,0,0
296,미용실 갔다온 이후로,며칠째 머가리에서 검은 가루가 나오는데 뭐냐 머리는 맨날감음,0,2
297,군무원학원다니는데,다음달 기본반끝나고 심화반 들어가거든 근데 선생님이 진지하게 공부계속할지 생각해보래...,0,2
298,ㅇㅅㅇ 머리 아파,1+2+3+4+5+. . .<1-2+3-4+5-6+. . .,0,3


# 크롤링 코드

# 크롤링 대상

# 1. 제목
# 2. 본문 내용
# 3. 댓글 
# 4. 시간
# 5. 좋아요 수

과정? 게시판 접근 후.
첫번째 게시글 
